In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
filepath = "../data/Final Categorization Data.csv"
df = pd.read_csv(filepath)

#drop extra binned columns for categorization models
df = df.drop(df.iloc[:,82:100], axis = 1)
df['Origin Destination New Region'] = df['Origin Province'] + df['New Regions']
#one hot encode remaining categorical variables
one_hot_columns = ["Main Transportation Method", "Quarter", "Main Reason" ]
df2 = pd.get_dummies(df, columns = one_hot_columns)
df2 = pd.get_dummies(df2, columns = ["Origin Destination New Region"])

In [9]:
#get list of column names for each feature
transport_cols = [col for col in df2.columns if ('Main Transportation Method') in col]
quarter_cols = [col for col in df2.columns if 'Quarter' in col]
mainreason_cols = [col for col in df2.columns if 'Main Reason' in col]
Origin_Destination_New_Region_cols = [col for col in df2.columns if 'Origin Destination New Region' in col]

#get dataframe subsets based on list of col names 
num_features = df2.loc[:,["DURATION", "GNCQ06A", "GNCQ06B"]]
transport = df2.loc[:,transport_cols]
quarter = df2.loc[:,quarter_cols]
main_reason = df2.loc[:,mainreason_cols]
ori_dest_newregion = df2.loc[:,Origin_Destination_New_Region_cols]
high_correlation_activities_cols =  df2.loc[:,['Visit Friends & Family', "Restaurant/Bar/Club", "Shopping", "Sightseeing", "Museum/Art Gallery", "Historic site", "Zoo/Aquarium", "Wildife Viewing/Bird Watching", "National/Provincial/Nature Park"]]

# same as above but dependent feature 
total_spend = df2.loc[:,["SPD_DTOT"]]

In [15]:
#consists of numerical inputs + ori dest new region, transport, transport, quarter, mainreason, high corr activities
x_list = [num_features, ori_dest_newregion, transport, quarter, main_reason, high_correlation_activities_cols]
# x3_list = [num_features, ori_dest_newregion, transport, quarter, main_reason, all_activities_cols]
x_list_transposed = [list_item.transpose() for list_item in x_list]
x = pd.concat(x_list_transposed, axis=0).transpose()
y = total_spend

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# model = LinearRegression().fit(X_train, y_train)
# y_pred = model.predict(X_test)

model = LinearRegression().fit(x, y)


In [7]:
import pickle
with open('linear_regression_model.pkl', 'wb') as file:
    pickle.dump(model, file)